# Requests et BeautifulSoup
## Installation Des Packages

In [1]:
#!pip install selenium

>**Remarque** : - pip est un gestionnaire de packages pour Python. Cela signifie que c'est un outil qui vous permet d'installer et de gérer des bibliothèques et des dépendances supplémentaires non distribuées dans le cadre de la bibliothèque standard. Assurez-vous de ne pas oublier **"!"** (comme ci-dessus) avant "pip install selenium", ce qui signifie l'exécuter comme une commande shell plutôt que comme une commande notebook (c'est comme si vous ouvriez un terminal et que vous le tapiez sans le **"!"**)

Nous lançons le navigateur Chrome lorsque nous utilisons Selenium. Nous devons installer un pilote (localement dans le système et exécuter ce notebook uniquement localement), qui agit comme un pont entre Selenium et Chrome (ou quel que soit le navigateur que vous utilisez). Pour Chrome, cela s'appelle le pilote Chrome, et pour Firefox, c'est le gecko.

In [2]:
from selenium import webdriver

Nous devons dire à Selenium quel navigateur nous utilisons, et Selenium pilotera le pilote Web.

In [3]:
#Si nous devons exécuter Selenium en mode sans entête ; c'est sans voir l'interface graphique en cours d'exécution, utilisez le code ci-dessous
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

In [4]:
driverPath_1 = 'C:\Program Files\Google\Chrome\Application\chromedriver.exe'
driverPath_2 = 'C:\Program Files\Google\Chrome\Application\chrome.exe'
serviceConfig = Service(driverPath_1)
optionConfig = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=serviceConfig, options=optionConfig)

Lorsque nous exécutons le code ci-dessus, chrome est automatisé, puis en gardant la page de message ouverte, nous utilisons la méthode get pour avoir l'URL de la page requise

In [5]:
items_url = "https://www.tayara.tn/"
driver.get(items_url) 

>**Remarque** : si nous exécutons la cellule de code ci-dessus sans exécuter la dernière cellule en priorité immédiate, le message d'erreur "**chrome inaccessible**" s'affichera.

Nous devons nous assurer que le navigateur affiche la page complète/les éléments complets de haut en bas (fin de page) comme nous, les humains, en faisant défiler vers le bas avec les valeurs des axes x et y. Et puis nous donnons un temps de sommeil de 4 secondes, donc nous ne rencontrons pas de problème où nous essayons de lire des éléments de la page, qui n'est pas encore chargée. Nous utilisons le code Selenium Execute pour cela. De plus, nous téléchargerons la fonction de temps pour ralentir un peu les choses/être doux sur le site Web.

In [6]:
import time

In [10]:
time.sleep(2) # dormir pendant deux secondes au début de la page, puis faire défiler vers le bas de la page 
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(4) #sleep pour charger tous les éléments

## Comprendre les méthodes de base de Selenium pour obtenir les données requises

Les techniques suivantes nous aideront à trouver des éléments dans une page Web (ces méthodes renverront une liste) :
<pre>
<code>
                find_elements_by_name
                find_elements_by_xpath
                find_elements_by_link_text
                find_elements_by_partial_link_text
                find_elements_by_tag_name
                find_elements_by_class_name
                find_elements_by_css_selector
</code>
</pre>

Écrivons maintenant un code Python pour récupérer des images sur le Web.

Nous utiliserons le sélecteur CSS et choisirons d'abord l'élément total, puis aborderons les valeurs individuelles.

Tous les éléments n'auront pas toutes les valeurs associées à cela, et certains peuvent également n'en avoir aucune. Nous utilisons donc chaque sélecteur CSS dans un bloc try-except qui fera quelque chose sauf pour aucun cas(**Exception en python**). Nous allons former une liste de dictionnaires, et nous pouvons stocker ces données dans un fichier JSON en utilisant le module [JSON](https://docs.python.org/3/library/json.html/) et la méthode [dump()](https://www.geeksforgeeks.org/json-dumps-in-python). Pour répéter le modèle précédent, nous le conserverons également dans un fichier CSV.

In [19]:
import time
import json
import os 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def get_items(search, page):
    search = search.replace(" ", "_")
    url = "https://www.tayara.tn/ads/c/Le_plus_r%C3%A9cent?price={}&p={}".format(
        search, page
    )
    driver.get(url)

    # sleep for two second then scroll to the bottom of the page
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4) #sleep between loadings

    items = driver.find_elements_by_css_selector(".mat-card")
    output = []

    for i in items:
        
        try:
            if len(i.find_element_by_css_selector(".title").text)>=1:
                titre = i.find_element_by_css_selector(".title").text
        except:
            titre = None

        try:
            if len(i.find_element_by_css_selector(".price").text)>=1:
                prix = i.find_element_by_css_selector(".price").text
        except:
            prix = None
            
        try:
            if len(i.find_element_by_css_selector(".mat-card-content a").text)>=1:
                produit_url = i.find_element_by_css_selector(".mat-card-content a").get_attribute("href")
        except:
            produit_url = None

        output_item = {"titre": titre, "prix": prix, "url": produit_url}
        output.append(output_item)

    return output

#elements-offer-price-normal__price
driverPath_1 = 'C:\Program Files\Google\Chrome\Application\chromedriver.exe'
serviceConfig = Service(driverPath_1)
optionConfig = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=serviceConfig, options=optionConfig)

page = 1
search = "0,9999999" #since we would like to search for Inflight Items
all_items = []
#we will call the function get_items
while True:
    print("getting page", page)
    results = get_items(search, page)
    all_items += results

    if len(results) == 0 or page > 2:
        break

    page += 1

# save all the items to a json file
json.dump(all_items, open("product.json", "w"), indent=2)

driver.close()

getting page 1


<ipython-input-19-4c39ffa89541>:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  items = driver.find_elements_by_css_selector(".mat-card")


getting page 2
getting page 3


In [20]:
results

[{'titre': '👉𝐂𝐀𝐘𝐄𝐍𝐍𝐄 𝟐𝟎𝟏𝟕💎𝐏𝐋𝐀𝐓𝐈𝐍𝐈𝐔𝐌 𝐄𝐃𝐈𝐓𝐈𝐎𝐍 V6❤️👌',
  'prix': '0 DT',
  'url': None},
 {'titre': '👉𝐂𝐀𝐘𝐄𝐍𝐍𝐄 𝟐𝟎𝟏𝟕💎𝐏𝐋𝐀𝐓𝐈𝐍𝐈𝐔𝐌 𝐄𝐃𝐈𝐓𝐈𝐎𝐍 V6❤️👌',
  'prix': '0 DT',
  'url': None},
 {'titre': '👉𝐂𝐀𝐘𝐄𝐍𝐍𝐄 𝟐𝟎𝟏𝟕💎𝐏𝐋𝐀𝐓𝐈𝐍𝐈𝐔𝐌 𝐄𝐃𝐈𝐓𝐈𝐎𝐍 V6❤️👌',
  'prix': '0 DT',
  'url': None},
 {'titre': '👉𝐂𝐀𝐘𝐄𝐍𝐍𝐄 𝟐𝟎𝟏𝟕💎𝐏𝐋𝐀𝐓𝐈𝐍𝐈𝐔𝐌 𝐄𝐃𝐈𝐓𝐈𝐎𝐍 V6❤️👌',
  'prix': '0 DT',
  'url': None},
 {'titre': '👉𝐂𝐀𝐘𝐄𝐍𝐍𝐄 𝟐𝟎𝟏𝟕💎𝐏𝐋𝐀𝐓𝐈𝐍𝐈𝐔𝐌 𝐄𝐃𝐈𝐓𝐈𝐎𝐍 V6❤️👌',
  'prix': '0 DT',
  'url': None},
 {'titre': '👉𝐂𝐀𝐘𝐄𝐍𝐍𝐄 𝟐𝟎𝟏𝟕💎𝐏𝐋𝐀𝐓𝐈𝐍𝐈𝐔𝐌 𝐄𝐃𝐈𝐓𝐈𝐎𝐍 V6❤️👌',
  'prix': '0 DT',
  'url': None},
 {'titre': '👉𝐂𝐀𝐘𝐄𝐍𝐍𝐄 𝟐𝟎𝟏𝟕💎𝐏𝐋𝐀𝐓𝐈𝐍𝐈𝐔𝐌 𝐄𝐃𝐈𝐓𝐈𝐎𝐍 V6❤️👌',
  'prix': '0 DT',
  'url': None},
 {'titre': '👉𝐂𝐀𝐘𝐄𝐍𝐍𝐄 𝟐𝟎𝟏𝟕💎𝐏𝐋𝐀𝐓𝐈𝐍𝐈𝐔𝐌 𝐄𝐃𝐈𝐓𝐈𝐎𝐍 V6❤️👌',
  'prix': '0 DT',
  'url': None},
 {'titre': '👉𝐂𝐀𝐘𝐄𝐍𝐍𝐄 𝟐𝟎𝟏𝟕💎𝐏𝐋𝐀𝐓𝐈𝐍𝐈𝐔𝐌 𝐄𝐃𝐈𝐓𝐈𝐎𝐍 V6❤️👌',
  'prix': '0 DT',
  'url': None},
 {'titre': '👉𝐂𝐀𝐘𝐄𝐍𝐍𝐄 𝟐𝟎𝟏𝟕💎𝐏𝐋𝐀𝐓𝐈𝐍𝐈𝐔𝐌 𝐄𝐃𝐈𝐓𝐈𝐎𝐍 V6❤️👌',
  'prix': '0 DT',
  'url': None},
 {'titre': '👉𝐂𝐀𝐘𝐄𝐍𝐍𝐄 𝟐𝟎𝟏𝟕💎𝐏𝐋𝐀𝐓𝐈𝐍𝐈𝐔𝐌 𝐄𝐃𝐈𝐓𝐈𝐎𝐍 V6❤️👌',
  'prix': '0 DT',
  'url': None},
 {'titre': '👉𝐂𝐀𝐘𝐄𝐍𝐍𝐄 𝟐𝟎𝟏𝟕💎𝐏𝐋𝐀𝐓𝐈𝐍𝐈𝐔𝐌 𝐄𝐃𝐈𝐓𝐈𝐎𝐍 V6❤️👌',
  